In [ ]:
from numpy import zeros, ones, expand_dims, asarray

from numpy.random import randn, randint

from keras.datasets import fashion_mnist

from keras.optimizers import Adam

from keras.models import Model, load_model

from keras.layers import Input, Dense, Reshape, Flatten

from keras.layers import Conv2D, Conv2DTranspose, Concatenate

from keras.layers import LeakyReLU, Dropout, Embedding

from keras.layers import BatchNormalization, Activation

from keras import initializers

from keras.initializers import RandomNormal

from keras.optimizers import Adam, RMSprop, SGD

from matplotlib import pyplot

import numpy as np

from math import sqrt

In [ ]:

(X_train, _), (_, _) = fashion_mnist.load_data()

X_train = X_train.astype(np.float32) / 127.5 - 1

X_train = np.expand_dims(X_train, axis=3)

print(X_train.shape)


4422102/4422102 [==============================] - 0s 0us/step
(60000, 28, 28, 1)


In [ ]:


def define_discriminator(in_shape=(28, 28, 1)):

    in_image = Input(shape=in_shape)

    fe = Flatten()(in_image)

    fe = Dense(1024)(fe)

    fe = LeakyReLU(alpha=0.2)(fe)

    fe = Dropout(0.3)(fe)

    fe = Dense(512)(fe)

    fe = LeakyReLU(alpha=0.2)(fe)

    fe = Dropout(0.3)(fe)

    fe = Dense(256)(fe)

    fe = LeakyReLU(alpha=0.2)(fe)

    fe = Dropout(0.3)(fe)

    out = Dense(1, activation='sigmoid')(fe)

    model = Model(in_image, out)

    opt = Adam(lr=0.0002, beta_1=0.5)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

discriminator = define_discriminator()

def define_generator(latent_dim):

    init  = RandomNormal(stddev=0.02) #generates an array of specified shapes and fills it with random values, which is actually a part of Normal (Gaussian)Distribution

    in_lat = Input(shape=(latent_dim,))

    gen = Dense(256, kernel_initializer=init)(in_lat)

    gen = LeakyReLU(alpha=0.2)(gen)

    gen = Dense(512, kernel_initializer=init)(gen)

    gen = LeakyReLU(alpha=0.2)(gen)

    gen = Dense(1024, kernel_initializer=init)(gen)

    gen = LeakyReLU(alpha=0.2)(gen)

    gen = Dense(28 * 28 * 1, kernel_initializer=init)(gen)

    out_layer = Activation('tanh')(gen)

    out_layer = Reshape((28, 28, 1))(gen)

    model = Model(in_lat, out_layer)

    return model

generator = define_generator(100)

def define_gan(g_model, d_model):

    d_model.trainable = False

    gan_output = d_model(g_model.output)

    model = Model(g_model.input, gan_output)

    opt = Adam(lr=0.0002, beta_1=0.5)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

gan_model = define_gan(generator, discriminator)



/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def generate_latent_points(latent_dim, n_samples):

    x_input = randn(latent_dim * n_samples)   #creates an array of specified shape and fills it with random values as per standard normal distribution

    z_input = x_input.reshape(n_samples, latent_dim)

    return z_input

def generate_real_samples(X_train, n_samples):

    ix = randint(0, X_train.shape[0], n_samples)  # returns an integer number selected element from the specified range.

    X = X_train[ix]

    y = ones((n_samples, 1))

    return X, y

def generate_fake_samples(generator, latent_dim, n_samples):

    z_input = generate_latent_points(latent_dim, n_samples)

    images = generator.predict(z_input)

    y = zeros((n_samples, 1))

    return images, y


#generating a fake sample, plotting it, and finally saving the model.

def summarize_performance(step, g_model, latent_dim, n_samples=100):

    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)

    X = (X + 1) / 2.0

    for i in range(100):

        pyplot.subplot(10, 10, 1 + i)

        pyplot.axis('off')

        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')

    filename2 = 'model_%04d.h5' % (step+1)

    g_model.save(filename2)

    print('>Saved: %s' % (filename2))


#plot the generated images by the  Generator in later stages.
def save_plot(examples, n_examples):

    for i in range(n_examples):

        pyplot.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i)

        pyplot.axis('off')

        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')

    pyplot.show()


In [ ]:
def train(g_model, d_model, gan_model, X_train, latent_dim, n_epochs=100, n_batch=64):

    bat_per_epo = int(X_train.shape[0] / n_batch)

    n_steps = bat_per_epo * n_epochs

    for i in range(n_steps):

        X_real, y_real = generate_real_samples(X_train, n_batch)

        d_loss_r, d_acc_r = d_model.train_on_batch(X_real, y_real)

        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)

        d_loss_f, d_acc_f = d_model.train_on_batch(X_fake, y_fake)

        z_input = generate_latent_points(latent_dim, n_batch)

        y_gan = ones((n_batch, 1))

        g_loss, g_acc = gan_model.train_on_batch(z_input, y_gan)

        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_loss_r,d_acc_r, d_loss_f,d_acc_f, g_loss,g_acc))

        if (i+1) % (bat_per_epo * 1) == 0:

            summarize_performance(i, g_model, latent_dim)

latent_dim = 100

train(generator, discriminator, gan_model, X_train, latent_dim, n_epochs=20, n_batch=64)

model = load_model('model_18740.h5')

latent_dim = 100

n_examples = 100

latent_points = generate_latent_points(latent_dim, n_examples)

X  = model.predict(latent_points)

X = (X + 1) / 2.0

save_plot(X, n_examples)

2/2 [==============================] - 0s 50ms/step
>1, dr[0.957,0.328], df[0.700,0.000], g[0.687,0.984]
2/2 [==============================] - 0s 17ms/step
>2, dr[0.119,1.000], df[0.713,0.000], g[0.675,1.000]
2/2 [==============================] - 0s 16ms/step
>3, dr[0.048,1.000], df[0.726,0.000], g[0.666,1.000]
2/2 [==============================] - 0s 36ms/step
>4, dr[0.024,1.000], df[0.745,0.000], g[0.651,1.000]
2/2 [==============================] - 0s 28ms/step
>5, dr[0.015,1.000], df[0.761,0.000], g[0.636,1.000]
2/2 [==============================] - 0s 22ms/step
>6, dr[0.012,1.000], df[0.793,0.000], g[0.614,1.000]
2/2 [==============================] - 0s 18ms/step
>7, dr[0.013,1.000], df[0.818,0.000], g[0.600,1.000]
2/2 [==============================] - 0s 30ms/step
>8, dr[0.011,1.000], df[0.843,0.000], g[0.595,1.000]
2/2 [==============================] - 0s 21ms/step
>9, dr[0.021,1.000], df[0.858,0.000], g[0.592,1.000]
2/2 [==============================] - 0s 21ms/step
>10

>Saved: model_0937.h5
2/2 [==============================] - 0s 11ms/step
>938, dr[0.570,0.734], df[0.566,0.750], g[2.531,0.031]
2/2 [==============================] - 0s 13ms/step
>939, dr[0.393,0.797], df[0.612,0.812], g[2.870,0.031]
2/2 [==============================] - 0s 11ms/step
>940, dr[0.502,0.703], df[0.509,0.812], g[2.864,0.031]
2/2 [==============================] - 0s 13ms/step
>941, dr[0.500,0.750], df[0.605,0.875], g[2.614,0.000]
2/2 [==============================] - 0s 11ms/step
>942, dr[0.325,0.812], df[0.380,0.891], g[2.676,0.047]
2/2 [==============================] - 0s 13ms/step
>943, dr[0.456,0.750], df[0.423,0.828], g[2.890,0.000]
2/2 [==============================] - 0s 10ms/step
>944, dr[0.486,0.734], df[0.374,0.828], g[2.495,0.031]
2/2 [==============================] - 0s 11ms/step
>945, dr[0.305,0.828], df[0.459,0.844], g[2.680,0.031]
2/2 [==============================] - 0s 12ms/step
>946, dr[0.371,0.781], df[0.405,0.750], g[2.995,0.000]
2/2 [==========

<ipython-input-4-7ddca67754e9>:37: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  pyplot.subplot(10, 10, 1 + i)


>Saved: model_1874.h5
2/2 [==============================] - 0s 12ms/step
>1875, dr[0.556,0.672], df[0.226,0.969], g[2.044,0.000]
2/2 [==============================] - 0s 12ms/step
>1876, dr[0.463,0.672], df[0.582,0.812], g[2.142,0.078]
2/2 [==============================] - 0s 11ms/step
>1877, dr[0.403,0.734], df[0.281,0.953], g[2.025,0.062]
2/2 [==============================] - 0s 13ms/step
>1878, dr[0.610,0.625], df[0.384,0.891], g[1.966,0.016]
2/2 [==============================] - 0s 11ms/step
>1879, dr[0.426,0.797], df[0.383,0.922], g[2.091,0.000]
2/2 [==============================] - 0s 12ms/step
>1880, dr[0.439,0.688], df[0.422,0.875], g[2.067,0.000]
2/2 [==============================] - 0s 12ms/step
>1881, dr[0.368,0.781], df[0.389,0.891], g[2.259,0.016]
2/2 [==============================] - 0s 10ms/step
>1882, dr[0.311,0.812], df[0.255,0.969], g[2.516,0.000]
2/2 [==============================] - 0s 14ms/step
>1883, dr[0.674,0.594], df[0.386,0.922], g[2.260,0.000]
2/2 [=

>Saved: model_2811.h5
2/2 [==============================] - 0s 14ms/step
>2812, dr[0.410,0.844], df[0.432,0.875], g[1.725,0.031]
2/2 [==============================] - 0s 12ms/step
>2813, dr[0.565,0.672], df[0.556,0.812], g[1.659,0.047]
2/2 [==============================] - 0s 13ms/step
>2814, dr[0.352,0.797], df[0.322,0.922], g[1.902,0.031]
2/2 [==============================] - 0s 18ms/step
>2815, dr[0.459,0.703], df[0.424,0.844], g[1.751,0.062]
2/2 [==============================] - 0s 14ms/step
>2816, dr[0.452,0.750], df[0.438,0.844], g[1.715,0.078]
2/2 [==============================] - 0s 13ms/step
>2817, dr[0.542,0.672], df[0.364,0.938], g[1.805,0.031]
2/2 [==============================] - 0s 21ms/step
>2818, dr[0.401,0.797], df[0.391,0.906], g[1.995,0.047]
2/2 [==============================] - 0s 16ms/step
>2819, dr[0.519,0.656], df[0.470,0.844], g[1.924,0.016]
2/2 [==============================] - 0s 19ms/step
>2820, dr[0.545,0.625], df[0.551,0.750], g[1.809,0.047]
2/2 [=

...

>Saved: model_5622.h5
2/2 [==============================] - 0s 16ms/step
>5623, dr[0.565,0.562], df[0.590,0.750], g[1.256,0.109]
2/2 [==============================] - 0s 13ms/step
>5624, dr[0.625,0.594], df[0.546,0.781], g[1.298,0.094]
2/2 [==============================] - 0s 11ms/step
>5625, dr[0.662,0.562], df[0.554,0.734], g[1.205,0.109]
2/2 [==============================] - 0s 10ms/step
>5626, dr[0.681,0.500], df[0.699,0.719], g[1.292,0.031]
2/2 [==============================] - 0s 13ms/step
>5627, dr[0.539,0.625], df[0.510,0.766], g[1.369,0.078]
2/2 [==============================] - 0s 15ms/step
>5628, dr[0.511,0.719], df[0.398,0.906], g[1.261,0.078]
2/2 [==============================] - 0s 17ms/step
>5629, dr[0.722,0.547], df[0.452,0.828], g[1.263,0.125]
2/2 [==============================] - 0s 18ms/step
>5630, dr[0.588,0.609], df[0.650,0.734], g[1.238,0.109]
2/2 [==============================] - 0s 13ms/step
>5631, dr[0.551,0.578], df[0.588,0.750], g[1.278,0.125]
2/2 [=

>Saved: model_6559.h5
2/2 [==============================] - 0s 15ms/step
>6560, dr[0.618,0.516], df[0.546,0.828], g[1.158,0.156]
2/2 [==============================] - 0s 18ms/step
>6561, dr[0.667,0.516], df[0.525,0.828], g[1.161,0.156]
2/2 [==============================] - 0s 13ms/step
>6562, dr[0.639,0.547], df[0.555,0.781], g[1.056,0.156]
2/2 [==============================] - 0s 12ms/step
>6563, dr[0.606,0.594], df[0.555,0.750], g[1.145,0.078]
2/2 [==============================] - 0s 14ms/step
>6564, dr[0.587,0.656], df[0.538,0.812], g[1.124,0.141]
2/2 [==============================] - 0s 13ms/step
>6565, dr[0.524,0.625], df[0.594,0.688], g[1.094,0.141]
2/2 [==============================] - 0s 11ms/step
>6566, dr[0.516,0.719], df[0.610,0.750], g[1.169,0.078]
2/2 [==============================] - 0s 12ms/step
>6567, dr[0.552,0.641], df[0.537,0.766], g[1.280,0.062]
2/2 [==============================] - 0s 15ms/step
>6568, dr[0.498,0.719], df[0.586,0.781], g[1.304,0.047]
2/2 [=

>Saved: model_7496.h5
2/2 [==============================] - 0s 12ms/step
>7497, dr[0.579,0.625], df[0.624,0.750], g[1.255,0.125]
2/2 [==============================] - 0s 12ms/step
>7498, dr[0.564,0.609], df[0.554,0.781], g[1.061,0.234]
2/2 [==============================] - 0s 33ms/step
>7499, dr[0.554,0.625], df[0.595,0.766], g[1.178,0.125]
2/2 [==============================] - 0s 11ms/step
>7500, dr[0.648,0.609], df[0.623,0.812], g[1.191,0.125]
2/2 [==============================] - 0s 12ms/step
>7501, dr[0.663,0.531], df[0.489,0.922], g[1.069,0.141]
2/2 [==============================] - 0s 16ms/step
>7502, dr[0.659,0.500], df[0.615,0.719], g[1.053,0.234]
2/2 [==============================] - 0s 21ms/step
>7503, dr[0.608,0.562], df[0.653,0.734], g[1.096,0.188]
2/2 [==============================] - 0s 17ms/step
>7504, dr[0.582,0.656], df[0.529,0.781], g[1.048,0.156]
2/2 [==============================] - 0s 12ms/step
>7505, dr[0.611,0.594], df[0.590,0.719], g[1.045,0.188]
2/2 [=

>Saved: model_8433.h5
2/2 [==============================] - 0s 21ms/step
>8434, dr[0.589,0.656], df[0.611,0.703], g[1.098,0.156]
2/2 [==============================] - 0s 16ms/step
>8435, dr[0.576,0.609], df[0.553,0.781], g[1.102,0.125]
2/2 [==============================] - 0s 12ms/step
>8436, dr[0.583,0.641], df[0.533,0.797], g[1.135,0.125]
2/2 [==============================] - 0s 12ms/step
>8437, dr[0.661,0.516], df[0.532,0.812], g[1.028,0.156]
2/2 [==============================] - 0s 18ms/step
>8438, dr[0.553,0.625], df[0.805,0.688], g[1.125,0.078]
2/2 [==============================] - 0s 21ms/step
>8439, dr[0.611,0.547], df[0.518,0.859], g[1.133,0.078]
2/2 [==============================] - 0s 16ms/step
>8440, dr[0.710,0.516], df[0.551,0.781], g[1.118,0.109]
2/2 [==============================] - 0s 10ms/step
>8441, dr[0.700,0.531], df[0.592,0.766], g[1.038,0.188]
2/2 [==============================] - 0s 13ms/step
>8442, dr[0.537,0.703], df[0.719,0.688], g[1.110,0.031]
2/2 [=

>Saved: model_9370.h5
2/2 [==============================] - 0s 14ms/step
>9371, dr[0.546,0.578], df[0.605,0.734], g[1.041,0.109]
2/2 [==============================] - 0s 13ms/step
>9372, dr[0.624,0.594], df[0.683,0.609], g[1.031,0.156]
2/2 [==============================] - 0s 13ms/step
>9373, dr[0.629,0.531], df[0.716,0.703], g[1.015,0.203]
2/2 [==============================] - 0s 13ms/step
>9374, dr[0.693,0.438], df[0.518,0.844], g[0.932,0.203]
2/2 [==============================] - 0s 11ms/step
>9375, dr[0.697,0.516], df[0.596,0.750], g[0.876,0.281]
2/2 [==============================] - 0s 18ms/step
>9376, dr[0.545,0.672], df[0.682,0.547], g[0.938,0.203]
2/2 [==============================] - 0s 13ms/step
>9377, dr[0.656,0.500], df[0.648,0.719], g[1.037,0.078]
2/2 [==============================] - 0s 13ms/step
>9378, dr[0.658,0.531], df[0.678,0.719], g[0.965,0.172]
2/2 [==============================] - 0s 13ms/step
>9379, dr[0.690,0.547], df[0.568,0.750], g[1.083,0.062]
2/2 [=